In [15]:
from backend.exploration_engine.exploration_engine import *
from backend.tree_navigator.tree_navigator import *
%load_ext autoreload
%autoreload 2

system_message = open("messages/system_message.md", "r").read()
user_message = open("messages/user_message.md", "r").read()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
tree = FolderNavigator("jpalioapp", "/Users/forna/Documents/me/palio")
model = "qwen2.5-coder-14b-instruct"
llm_client = OpenAILLMClient(base_url="http://localhost:1234", model_name=model, api_key="")
objective = "Find the game rest controller file/s"

In [17]:
def summarize(node_path):
    text = tree.open(node_path)
    tree.close(node_path)
    node_name = node_path.split("/")[-1]
    system_message = """
You are an expert system designed to help developers.
"""
    user_message = f"""
Provide a very brief summary of the content of the file below.
The summary will be used to help another system to understand if the file is relevant to the objective.
The objective is: {objective}
Write the summary inside a code block with the name of the file as the node name.

```{node_name}
{text}
```
"""
    messages = [system_message, user_message]
    response = llm_client.chat_completion(messages)
    response = response["choices"][0]["message"]["content"]
    response = response.split("```")[1]

    return response

In [11]:
actions = [
    {
        "name": "open",
        "description": "Open the folder and updates the tree with the children and their metadata",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file or folder"
            }
        },
        "__run__": tree.open
    },
    {
        "name": "close",
        "description": "Close the folder and remove the children from the tree",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file or folder"
            }
        },
        "__run__": tree.close
    },
    {
        "name": "open_file",
        "description": "Open the file and save the content in the tree",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file"
            }
        },
        "comments": [
            "Always check the file size before opening it. If the file is too large, consider summarizing it"
        ],
        "__run__": tree.open
    },
    {
        "name": "close_file",
        "description": "Close the file and remove the content from the tree",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file"
            }
        },
        "__run__": tree.close
    },
    {
        "name": "summarize",
        "description": "Summarize the content of the file and save it in the tree",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file"
            }
        }

    },
    {
        "name": "mark_useful",
        "description": "Mark the file or folder as useful",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file or folder"
            }
        }
    },
    {
        "name": "mark_useless",
        "description": "Mark the file or folder as useless",
        "parameters": {
            "node": {
                "type": "string",
                "description": "The full path of the file or folder"
            }
        }
    }
]

In [22]:
model = "qwen2.5-coder-14b-instruct"
llm_client = OpenAILLMClient(base_url="http://localhost:1234", model_name=model, api_key="")

system_message = """
You are an expert system designed to help developers.
"""
text = open("../tree_navigator/tree_navigator.py").read()
user_message = f"""
Provide a brief descriptive summary of the content of the file below.
The summary will be used to help another system to understand if the file is relevant to the objective.
The objective is: {objective}

```tree_navigator.py
{text}
```
"""

print(user_message)
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
response = llm_client.chat_completion(messages)
response = response["choices"][0]["message"]["content"]
response = response.split("```")[1]


Provide a very brief summary of the content of the file below.
The summary will be used to help another system to understand if the file is relevant to the objective.
The objective is: Find the game rest controller file/s

```tree_navigator.py
import json
import os
from abc import ABC


class Node:
    def __init__(self, name: str, content, **attributes):
        self.name = name
        self.content = content
        self.__dict__.update(attributes)

    def set_attribute(self, key, value):
        self.__dict__[key] = value

    def get_attribute(self, key):
        return self.__dict__.get(key)

    def __setitem__(self, key, value):
        self.__dict__[key] = value

    def get_attributes(self):
        return self.__dict__.keys() - ["name", "content"]


class TreeNavigator(ABC):
    def __init__(self, root_name: str):
        self.tree = Node(root_name, None)

    def get_node(self, node_path) -> Node | None:
        """
        Get the node object from the tree
        :param 

IndexError: list index out of range